In [46]:
import requests # make a request to retrieve the page
from bs4 import BeautifulSoup # html parser
import pandas as pd

## Make Connections

In [2]:
base_url = "https://hotels.ng/hotels-in-abia"
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36q (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"}

print("Connecting..")
response_page = requests.get(base_url, headers=header)

if response_page.status_code == 200:
    print("Connected Successfully")
else:
    print("Connection not succesful!")

Connecting..
Connected Successfully


## Parse the pages and begin scraping

In [10]:
# To parse the html page...
parsed_page = BeautifulSoup(response_page.text, 'html.parser')

hotel_listing = parsed_page.find_all('div', class_="listing-hotels")

In [45]:
all_hotels_listing = []
for listing in hotel_listing:
    hotel_name = listing.find('h2', {"class": "listing-hotels-name"}).text
    hotel_address = listing.find('p', class_="listing-hotels-address color-dark").text
    hotel_address = " ".join(hotel_address.split())
    address = hotel_address.split(" - ")[1]
    city = hotel_address.split(" - ")[0].split(',')[0]
    state = hotel_address.split(" - ")[0].split(',')[1].strip()
    price = listing.find('p', class_="listing-hotels-prices-discount").text
    price = price.split()[0].replace("₦", "").replace(",", "")
    rated = listing.find("p", "listing-hotels-rating")
    if rated is None:
        rating = 0.0
        index = "No Index"
    else:
        rating = rated.text.split(" - ")[0]
        index = rated.text.split(" - ")[1]
    facility = listing.find("div", class_="listing-hotels-facilities d-none d-md-flex")
    if facility is None:
        facilities = "No Facilities Available"
    else:
        facilities = facility.find_all('div', class_="d-flex flex-column align-items-center")
        facilities = [fac.find('p').text for fac in facilities]
        facilities = ", ".join(facilities)
    likes = listing.find('div', class_="listing-hotels-likes").text
    likes = likes.strip().split()[0]
    
    # temp storage
    hotels_listing = {
        "Hotel_Name": hotel_name,
        "Hotel_Address": address,
        "City": city,
        "State": state,
        "Price_Per_Night": price,
        "Rating_Score": rating,
        "Rating_Index": index,
        "Facilities": facilities,
        "Popularity": likes
    }
    all_hotels_listing.append(hotels_listing)


[{'Hotel_Name': 'Luxury City Royal', 'Hotel_Address': '84b, Aba-owerri Road', 'City': 'Aba', 'State': 'Abia', 'Price_Per_Night': '12500', 'Rating_Score': '5.8', 'Rating_Index': 'Good', 'Facilities': 'Bar, Gym, Restaurant, Pool, Internet', 'Popularity': '2,618'}, {'Hotel_Name': 'Hotel Royal Damgrete', 'Hotel_Address': '1314 Factory Road, Gra', 'City': 'Umuahia', 'State': 'Abia', 'Price_Per_Night': '35000', 'Rating_Score': '7', 'Rating_Index': 'Very Good', 'Facilities': 'Bar, Restaurant, Pool, Internet', 'Popularity': '8,003'}, {'Hotel_Name': 'City Global Hotels', 'Hotel_Address': '32 Ekenna Avenue Off Okigwe Road', 'City': 'Aba', 'State': 'Abia', 'Price_Per_Night': '9016', 'Rating_Score': '6.2', 'Rating_Index': 'Very Good', 'Facilities': 'Bar, Restaurant, Pool', 'Popularity': '2,106'}, {'Hotel_Name': 'De Merit Executive Suite ', 'Hotel_Address': '2 Amaimo Eziama Ossa, Umuahia, Nigeria.', 'City': 'Umuahia', 'State': 'Abia', 'Price_Per_Night': '8000', 'Rating_Score': '5.1', 'Rating_Index'

In [47]:
df_hotel = pd.DataFrame(all_hotels_listing)
df_hotel.head(7)

,Hotel_Name,Hotel_Address,City,State,Price_Per_Night,Rating_Score,Rating_Index,Facilities,Popularity
0,Luxury City Royal,"84b, Aba-owerri Road",Aba,Abia,12500,5.8,Good,"Bar, Gym, Restaurant, Pool, Internet","2,618"
1,Hotel Royal Damgrete,"1314 Factory Road, Gra",Umuahia,Abia,35000,7,Very Good,"Bar, Restaurant, Pool, Internet","8,003"
2,City Global Hotels,32 Ekenna Avenue Off Okigwe Road,Aba,Abia,9016,6.2,Very Good,"Bar, Restaurant, Pool","2,106"
3,De Merit Executive Suite,"2 Amaimo Eziama Ossa, Umuahia, Nigeria.",Umuahia,Abia,8000,5.1,Good,"Bar, Restaurant","4,605"
4,Ibiza Hotel & Resorts,"129 Okigwe Road, Aba",Aba,Abia,55000,0.0,No Index,"Bar, Gym, Restaurant, Pool","2,410"
5,Villa Roy Hotels,19 Mission Hill,Umuahia,Abia,12000,7.8,Very Good,Bar,"2,402"
6,Craycaston Hotels & Suites,"19 Nwaigwe Ave, Aba 450272, Aba",Aba,Abia,35200,0.0,No Index,"Bar, Restaurant, Pool","3,818"
